In [1]:
import pandas as pd
import numpy as np
import os
import tqdm
import shutil

In [13]:
dataset = 'session_emma' # onion or emma, session_onion or session_emma
input_data_dir = f'../data/'

features = {
    'emotion_embeddings': ['id_emma.tsv', 'id_gems.tsv'],
    'audio_embeddings': ['id_maest.tsv', 'id_jukebox.tsv', 'id_musicnn.tsv'],
    'textual_embeddings': ['id_bert.tsv', 'id_lyrics_word2vec.tsv'],
    'visual_embeddings': ['id_resnet.tsv', 'id_vgg19.tsv', 'id_incp.tsv'],
}

output_dir = f'../dataset/{dataset}/'

os.makedirs(output_dir, exist_ok=True)

In [14]:
df_interactions = pd.read_csv(f'../data/userid_trackid_timestamp_{dataset}.tsv', sep='\t')
    
df_interactions['rating'] = 5
sample_items = df_interactions['track_id'].unique()

# map item and users to numbers
df_interactions['user_id_int'] = df_interactions['user_id'].astype('category').cat.codes
df_interactions['track_id_int'] = df_interactions['track_id'].astype('category').cat.codes


print(df_interactions.shape)
df_interactions.head()

(231568, 7)


,user_id,track_id,timestamp,session_id,rating,user_id_int,track_id_int
0,6,DMSPSXCv8UN5942f,2011-10-17 02:22:02,115,5,0,48
1,6,lJKIbZNzpS6IsNgh,2011-10-17 02:46:05,115,5,0,177
2,9,yWrNTxHepxJrplD1,2009-12-09 15:34:45,216,5,1,231
3,9,zbI3TrMSY2chqzRz,2009-12-09 15:47:16,216,5,1,234
4,16,cCmrNVfqxEnJiKrQ,2008-11-10 12:12:30,637,5,2,146


In [15]:
# track ids from feature file
df = pd.read_csv(input_data_dir + features['emotion_embeddings'][1], sep='\t')
feat_track_ids = df['id'].values
feat_track_ids

array(['04q3VppIQEET5rzy', '07xF9Q0K1t3ist7K', '0JbYcELIqpMPvAHk', ...,
       'zzyyPUs7hC9Nz2e1', 'zzz0n04uuTUA7fNh', 'zzzj3LYaZtYtbzSr'],
      dtype=object)

In [16]:
# remove interactions with items not in the feature files
df_interactions = df_interactions[df_interactions['track_id'].isin(feat_track_ids)]
df_interactions.shape

(231568, 7)

In [17]:
#df_interactions[['user_id', 'track_id', 'rating', 'timestamp']].to_csv(os.path.join(output_dir, 'interactions.tsv'), index=False, sep='\t', header=False)
df_interactions[['user_id_int', 'track_id_int', 'rating', 'timestamp']].to_csv(os.path.join(output_dir, 'interactions.tsv'), index=False, sep='\t', header=False)

In [18]:
item_id_map = df_interactions[['track_id', 'track_id_int']].drop_duplicates()
item_id_map.set_index('track_id', inplace=True)
item_id_map.head()

,track_id_int
track_id,
DMSPSXCv8UN5942f,48
lJKIbZNzpS6IsNgh,177
yWrNTxHepxJrplD1,231
zbI3TrMSY2chqzRz,234
cCmrNVfqxEnJiKrQ,146


In [19]:
del df_interactions

In [20]:
for feature_type in features:

    for file in features[feature_type]:
        if dataset == 'onion' and 'emma' in file:
            continue

        print('Processing', file)
        df = pd.read_csv(os.path.join(input_data_dir, file), index_col=0, sep='\t')
        df = df.merge(item_id_map, left_index=True, right_index=True)
        feature_name = file.split('.')[0].split('_')[1]
            
        folder_path = os.path.join(output_dir, feature_type, feature_name)

        if os.path.exists(folder_path):
            shutil.rmtree(folder_path)
        os.makedirs(folder_path, exist_ok=True)

        for index, row in tqdm.tqdm(df.iterrows()):
            np.save(os.path.join(folder_path, str(int(row['track_id_int'])) + '.npy'), row.values[:-1])

Processing id_emma.tsv


238it [00:01, 216.86it/s]


Processing id_gems.tsv


238it [00:01, 220.26it/s]


Processing id_maest.tsv


...

238it [00:01, 168.83it/s]


Processing id_vgg19.tsv


238it [00:01, 133.77it/s]


Processing id_incp.tsv


238it [00:01, 170.21it/s]
